## Chaos Base Class definition

In [ ]:
import numpy as np

class ChaosGen () :    
######################################################################
# If cascade=False, then each point in the (Np, D) matrix evolves
# independently of other points according to the map. For the CPSO, this
# amounts to having a certain correlation between the random numbers r1, r2
# per iteration of the CPSO
#
# If cascade=True, then each of the Np particles is connected to the
# previous one via the chaotic map. Every dimension is independent of the
# other, however!
######################################################################
    
    def __init__ (self, oshape, gshape=None, cascade=True, gens=2) :
        self.oshape = oshape
        self.gshape = (lambda s: s[1:] if cascade else s)(oshape if gshape is None else gshape)
        self.cascade = cascade
        self.gens = gens
        self.cgens = (lambda s:np.array([np.random.random_sample (s) for i in range(gens)]))\
                    (self.gshape)
      
    def getCgens (self) :
        return self.cgens
                 
    def chaosPoints (self, gno=0) :
        if gno != 0 :
            if self.cascade :
                return np.array ([
                    self.evolve(gno-1) for i in range(self.oshape[0])
                ])
            else :
                return self.evolve(gno-1)
        else :
            return np.array ([
                self.chaosPoints (i+1) for i in range(self.gens)
            ])
    
    def evolve (self) :
        # Defined in the specific chaotic classes
        pass

## Logistic Class

In [ ]:
class Logistic (ChaosGen) :
    def __init__ (self, oshape, cascade=True, r=4, gens=2) :
        ChaosGen.__init__(self, oshape, None, cascade, gens)
        self.r = r
        
    def evolve (self, gind) :
        ret = np.copy (self.cgens[gind])
        self.cgens[gind] = (lambda r,x : r*x*(1-x))(self.r, self.cgens[gind])
        return ret

## Tent Class

In [ ]:
class Tent (ChaosGen) :
    def __init__ (self, oshape, cascade=True, mu=0.49999, gens=2) :
        ChaosGen.__init__(self, oshape, None, cascade, gens)
        self.mu = mu
        
    def evolve (self, gind) :
        ret = np.copy (self.cgens[gind])
        self.cgens[gind] = (lambda mu,x : np.where(x <= mu, x/mu, (1-x)/(1-mu)))\
                            (self.mu, self.cgens[gind])
        return ret

## Lorenz Class

In [ ]:
from scipy.integrate import odeint

class Lorenz (ChaosGen) :
    lims = {}
    
    def lorenz (X, t, sigma, beta, rho) :
        x, y, z = X
        dXdt = [sigma*(y-x), x*(rho-z) - y, x*y - beta*z]
        return dXdt
    
    def setLimits (params) :
        if params not in Lorenz.lims :
            Lorenz.lims[params] = (lambda s:np.array([
                [np.min(s[:,i]), np.max(s[:,i])] for i in [0, 1, 2]
            ]))\
            (odeint (Lorenz.lorenz, np.random.rand(3), np.linspace (0, 9999, 999999), args = params))
            
            
    def __init__ (self, oshape, cascade=True, params=(10, 8.0/3, 28), comp=0, h=0.01, gens=2) :
        ChaosGen.__init__ (self, oshape, oshape+(3,), cascade, gens)
        self.params = params
        self.comp = comp
        self.h = h
        Lorenz.setLimits (params)
        
        for i in range(0, self.gens) :
            for j in [0, 1, 2] :
                self.cgens[i,...,j] = (lambda st,mn,mx : mn + (mx - mn)*st)\
                                (self.cgens[i,...,j], Lorenz.lims[params][j,0], Lorenz.lims[params][j,1])
        
    def evolveT (self, gind, T=1) :
        for pt in np.ndindex(self.gshape[:-1]) :
            self.cgens[gind][pt] = odeint(Lorenz.lorenz, self.cgens[gind][pt], 
                                          np.arange(0,self.h*(T+1),self.h), args=self.params)[-1]
        
    def evolve (self, gind) :
        eps = 1e-5
        ret = (lambda n2 : np.where (n2 > 1, 1-eps, n2))(
            (lambda n1 : np.where (n1 < 0, eps, n1))(
            (lambda st, mn, mx : (st - mn)/(mx - mn))
            (self.cgens[gind,...,self.comp], 
             Lorenz.lims[self.params][self.comp,0], 
             Lorenz.lims[self.params][self.comp,1])
            ))
        
        self.evolveT (gind)
        return ret

## Testing Chaos Generators

In [ ]:
# cg = Tent ((4,5), False)
cg = Lorenz ((4,5))


print (cg.oshape)
print (cg.gshape)
print ("-------------------------------------------------------")
print ((cgens := cg.getCgens()).shape)
print ((cgens0 := cg.chaosPoints()).shape)
print ((cgens1 := cg.chaosPoints(1)).shape)
print ("-------------------------------------------------------")

print (cgens)
print (cgens0)
print (cgens1)


print (Lorenz.lims[(10, 8/3, 28)])

pass

In [ ]:
print (Lorenz.lims[(10, 8/3, 28)])